<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#PageRank" data-toc-modified-id="PageRank-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>PageRank</a></span><ul class="toc-item"><li><span><a href="#加载数据" data-toc-modified-id="加载数据-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>加载数据</a></span></li><li><span><a href="#迭代算法" data-toc-modified-id="迭代算法-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>迭代算法</a></span></li></ul></li><li><span><a href="#存储" data-toc-modified-id="存储-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>存储</a></span><ul class="toc-item"><li><span><a href="#应用" data-toc-modified-id="应用-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>应用</a></span><ul class="toc-item"><li><span><a href="#查询" data-toc-modified-id="查询-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>查询</a></span></li><li><span><a href="#top-100" data-toc-modified-id="top-100-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>top 100</a></span></li></ul></li></ul></li></ul></div>

# PageRank

## 加载数据

In [3]:
import wiki
import imp
imp.reload(wiki)
wiki_titles = wiki.WikiTitles(file = "pass1.5.data")
print(wiki_titles.get_index_from_title("Anarchism"))
print(wiki_titles.get_title_from_index(wiki_titles.get_index_from_title("Anarchism")))

0
Anarchism


In [4]:
wiki_links = wiki.WikiLinks(file = "pass2.npz", wiki_titles=wiki_titles)
print(wiki_links.get_links_from_title("PageRank"))

['Logarithmic scale', 'Algorithm', 'Google Search', 'Web page', 'Web search engine', 'Larry Page', 'Google Patents', 'Network theory', 'Weighting', 'Hyperlink', 'Set (abstract data type)', 'World Wide Web', 'Link building', 'Webgraph', 'CNN', 'USA.gov', 'Recursion', 'Backlink', 'HITS algorithm', 'Jon Kleinberg', 'Teoma', 'Ask.com', 'CLEVER project', 'TrustRank', 'Google Hummingbird', 'Eigenvalues and eigenvectors', 'Scientometrics', 'Thomas L. Saaty', 'Analytic hierarchy process', 'Cognitive model', 'Baidu', 'Robin Li', 'The New York Times', 'Forbes', 'Sergey Brin', 'Stanford University', 'Rajeev Motwani', 'Terry Winograd', 'Google', 'Software patent', 'Citation analysis', 'Eugene Garfield', 'Hyper Search', 'Massimo Marchiori', 'University of Padua', 'Probability distribution', 'Matt Cutts', 'Markov chain', 'URL', 'Adjacency matrix', 'Stochastic matrix', 'Eigenvector centrality', 'Eigengap', 'Expected value', 'Wikipedia', 'Link farm', 'Trade secret', 'Power iteration', 'Steady state', 

## 迭代算法

In [5]:
import numba as nb
import numpy as np
@nb.njit
def PR_update(links, page_n_links, PR, PR_out, d):
    
    n_pages = len(PR)
    
    for i in range(n_pages):
        PR_out[i] = 0.0
        
    l = 0
    for i in range(n_pages):
        
        n_links = page_n_links[i]
        # this let the sum of the PR smaller than 1
        if n_links > 0:
            w =  PR[i] / n_links

            for j in range(n_links):
                PR_out[links[l]] += w
                l += 1
        else:
#             PR_out[i] += PR[i]
            pass
            
    # damp
    bias = (1. - d)/n_pages
    for i in range(n_pages):
        PR_out[i] = PR_out[i]*d + bias
        
def page_rank(links, page_n_links, n_iters, d=0.85, on_update=None):

    PR = np.ones(len(page_n_links))/len(page_n_links)
    PR_out = np.empty(len(page_n_links))

    for i in range(n_iters):
        PR_update(links, page_n_links, PR, PR_out, d)

        if on_update:
            on_update(i, PR, PR_out)

        # swap
        t = PR
        PR = PR_out
        PR_out = t

    return PR

In [6]:
def on_update(i, PR, PR_new):
    if i % 10 == 0:
        norm = np.linalg.norm(PR - PR_new)
        print("%3d: %f   "%(i, norm), end="\n", flush=True)

PR = page_rank(wiki_links.links, wiki_links.page_n_links, on_update=on_update, n_iters=40)
PR = PR*len(PR)

  0: 0.009104   
 10: 0.000013   
 20: 0.000001   
 30: 0.000000   


# 存储

In [13]:
import pickle
with open("page_rank.data", "wb") as f:
    pickle.dump(PR, f)

In [9]:
import sqlite3
import os

if os.path.exists("page_rank.db"):
    os.remove("page_rank.db")
    
try:
    conn = sqlite3.connect("page_rank.db")
    c = conn.cursor()
    c.execute("create table page_rank (idx integer unique primary key, PageRank real)")
    
    for idx, pr in enumerate(PR):
        c.execute("insert into page_rank values (?, ?)", (idx, pr))
        if idx % 10000 == 0:
            conn.commit()
        if idx % 100000 == 0:
            print("\r%d            "%idx, end="")
    
finally:
    if conn:
        conn.close()

6000000            

## 应用

### 查询

In [5]:
def get_PR(title):
    index = wiki_titles.get_index_from_title(title)
    if index is not None and idx > 0 and idx < len(PR):
        return PR[index]
    else:
        return np.nan

In [6]:
regions = ["China", "Beijing", "Hongkong", "Shanghai", "Taiwan", "Guangdong", "Hebei", "Taipei", "Shijiazhuang", "Shenzhen", "USA"]
for region in regions:
    print("%-20s"%region, "%.1f"%get_PR(region))

China                2922.7
Beijing              726.1
Hongkong             960.5
Shanghai             405.5
Taiwan               962.3
Guangdong            221.3
Hebei                127.7
Taipei               226.6
Shijiazhuang         28.7
Shenzhen             106.0
USA                  10132.7


### top 100

In [7]:
def top100(PR):
    n = 0
    PR_I_SORT = np.argsort(PR)[::-1]
    for j in PR_I_SORT:
        title = wiki_titles.get_title_from_index(j)
        print("%-100s"%title, "%.1f"%PR[j])
        n += 1
        if n == 100:
            break
    
top100(PR)

United States                                                                                        10132.7
World War II                                                                                         5060.9
France                                                                                               4984.0
Association football                                                                                 4720.8
United Kingdom                                                                                       4572.3
List of sovereign states                                                                             4529.6
The New York Times                                                                                   4267.5
Germany                                                                                              4254.5
India                                                                                                4072.7
New York City              